In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import plotly.graph_objects as go
import pytz
pd.options.plotting.backend = "plotly"



In [18]:
# Load all gbtc data
df_futu_minute_gbtc_2020_06_01_2021_05_06 = pd.read_csv("../data/raw/futu_gbtc_minute_2020-06-01_2021-05-06.csv", sep=",")
# df_futu_minute_gbtc_2020_06_01_2021_05_06['date'] = df_futu_minute_gbtc_2020_06_01_2021_05_06['date_edt'].apply(lambda d: )
df_futu_minute_gbtc_2020_06_01_2021_05_06['date_edt'].astype('datetime64[ns]')

# df_futu_gbtc_minute = pd.concat([df_futu_minute_gbtc_2020_06_01_2021_05_06, xxxx]).drop_duplicates().sort_index(ascending=True)
df_futu_gbtc_minute = df_futu_minute_gbtc_2020_06_01_2021_05_06

eastern=pytz.timezone('US/Eastern')
utc=pytz.utc
df_futu_gbtc_minute['date'] = df_futu_gbtc_minute['date_edt'].apply(lambda d: (eastern.localize(datetime.strptime(d,"%Y-%m-%d %H:%M:%S"), is_dst=None)).astimezone(utc))
df_futu_gbtc_minute = df_futu_gbtc_minute.set_index('date')
df_futu_gbtc_minute.index.astype('datetime64[ns]')
# df_futu_gbtc_minute

# Load all btc data
df_minute_btc_usd_minute_2020_06_01_2021_04_18 = pd.read_csv("../data/raw/btc-usd_minute_2020-06-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2020_06_01_2021_04_18.index.astype('datetime64[ns]')
df_minute_btc_usd_minute_2021_04_18_2021_05_01 = pd.read_csv("../data/raw/btc-usd_minute_2021-04-18_2021-05-01.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2021_04_18_2021_05_01.index.astype('datetime64[ns]')

df_btc_minute = pd.concat([df_minute_btc_usd_minute_2020_06_01_2021_04_18, df_minute_btc_usd_minute_2021_04_18_2021_05_01]).drop_duplicates().sort_index(ascending=True)
df_btc_minute.index = pd.to_datetime(df_btc_minute.index, utc = True)
df_btc_minute.index.astype('datetime64[ns]')
df_btc_minute = df_btc_minute.sort_index(ascending=True)



In [21]:
# Get GBTC-BTC
df_minute = pd.merge(df_btc_minute, df_futu_gbtc_minute, on="date", how="left")

df_minute = df_minute.rename(columns = {'low_price':'btc_low_price', 'high_price':'btc_high_price', 'close_price':'btc_close_price', 'open_price':'btc_open_price', 'volume_x':'btc_volume', 'time':'time', 'date_edt':'gbtc_date_edt', 'open':'gbtc_open_price', 'high':'gbtc_high_price', 
'low':'gbtc_low_price', 'close':'gbtc_close_price', 'volume_y':'gbtc_volume'})

btc_per_share = (0.00094509 + 0.000944257 + 0.000943585) / 3 

df_minute["nav_open_price"] = df_minute["btc_open_price"] * btc_per_share
df_minute["nav_close_price"] = df_minute["btc_close_price"] * btc_per_share
df_minute["nav_high_price"] = df_minute["btc_high_price"] * btc_per_share
df_minute["nav_low_price"] = df_minute["btc_low_price"] * btc_per_share

df_minute["premium"] = (df_minute["gbtc_close_price"] - df_minute["nav_close_price"]) / df_minute["nav_close_price"]
df_minute["premium_high"] = (df_minute["gbtc_high_price"] - df_minute["nav_high_price"]) / df_minute["nav_high_price"]
df_minute["premium_open"] = (df_minute["gbtc_open_price"] - df_minute["nav_open_price"]) / df_minute["nav_open_price"]
df_minute["premium_close"] = (df_minute["gbtc_close_price"] - df_minute["nav_close_price"]) / df_minute["nav_close_price"]
df_minute["premium_low"] = (df_minute["gbtc_low_price"] - df_minute["nav_low_price"]) / df_minute["nav_low_price"]
# df_minute = df_minute.dropna()
df_minute.to_csv("../data/btc_gbtc/futu_btc_gbtc_minute_2021.csv")

In [27]:
# Get OBTC-BTC

df_futu_minute_obtc_2021_01_01_2021_05_06 = pd.read_csv("../data/raw/futu_obtc_minute_2021-01-01_2021-05-06.csv", sep=",")
df_futu_minute_obtc_2021_01_01_2021_05_06['date_edt'].astype('datetime64[ns]')

# df_futu_gbtc_minute = pd.concat([df_futu_minute_gbtc_2020_06_01_2021_05_06, xxxx]).drop_duplicates().sort_index(ascending=True)
df_futu_obtc_minute = df_futu_minute_obtc_2021_01_01_2021_05_06

eastern=pytz.timezone('US/Eastern')
utc=pytz.utc
df_futu_obtc_minute['date'] = df_futu_obtc_minute['date_edt'].apply(lambda d: (eastern.localize(datetime.strptime(d,"%Y-%m-%d %H:%M:%S"), is_dst=None)).astimezone(utc))
df_futu_obtc_minute = df_futu_obtc_minute.set_index('date')
df_futu_obtc_minute.index.astype('datetime64[ns]')

# df_futu_obtc_minute
df_obtc_minute = pd.merge(df_btc_minute, df_futu_obtc_minute, on="date", how="left")

df_obtc_minute = df_obtc_minute.rename(columns = {'low_price':'btc_low_price', 'high_price':'btc_high_price', 'close_price':'btc_close_price', 'open_price':'btc_open_price', 'volume_x':'btc_volume', 'time':'time', 'date_edt':'obtc_date_edt', 'open':'obtc_open_price', 'high':'obtc_high_price', 
'low':'obtc_low_price', 'close':'obtc_close_price', 'volume_y':'obtc_volume'})

btc_per_share = 0.000341 

df_obtc_minute["nav_open_price"] = df_obtc_minute["btc_open_price"] * btc_per_share
df_obtc_minute["nav_close_price"] = df_obtc_minute["btc_close_price"] * btc_per_share
df_obtc_minute["nav_high_price"] = df_obtc_minute["btc_high_price"] * btc_per_share
df_obtc_minute["nav_low_price"] = df_obtc_minute["btc_low_price"] * btc_per_share

df_obtc_minute["premium"] = (df_obtc_minute["obtc_close_price"] - df_obtc_minute["nav_close_price"]) / df_obtc_minute["nav_close_price"]
df_obtc_minute["premium_high"] = (df_obtc_minute["obtc_high_price"] - df_obtc_minute["nav_high_price"]) / df_obtc_minute["nav_high_price"]
df_obtc_minute["premium_open"] = (df_obtc_minute["obtc_open_price"] - df_obtc_minute["nav_open_price"]) / df_obtc_minute["nav_open_price"]
df_obtc_minute["premium_close"] = (df_obtc_minute["obtc_close_price"] - df_obtc_minute["nav_close_price"]) / df_obtc_minute["nav_close_price"]
df_obtc_minute["premium_low"] = (df_obtc_minute["obtc_low_price"] - df_obtc_minute["nav_low_price"]) / df_obtc_minute["nav_low_price"]
# df_minute = df_minute.dropna()
df_obtc_minute = df_obtc_minute['2021-03-15': ]
df_obtc_minute.to_csv("../data/btc_gbtc/futu_btc_obtc_minute_2021.csv")